In [14]:
import geopandas as gpd
import osmnx as ox
import censusdata
import json
%matplotlib inline

In [2]:
year=2017
state_fips=25

In [3]:
# bg=gpd.read_file('https://www2.census.gov/geo/tiger/TIGER{}/BG/tl_{}_{}_bg.zip'.format(year, year, state_fips))
# Convert to WGS84
# bg = bg.to_crs("EPSG:4326")

# Compute example metrics over walkable community

### Load json file which maps BGs to walkable BGs

In [19]:
bg_to_walkable_bg=json.load(open('bg_to_walkable_bg_osrm_{}.json'.format(state_fips)))

### Get the LODES Data

OD Files

In [20]:
import urllib.request as ur
from gzip import GzipFile
import pandas as pd

def get_od_data(state='ma'):
    req = ur.Request('https://lehd.ces.census.gov/data/lodes/LODES7/{}/od/ma_od_main_JT00_2017.csv.gz'.format(state)) 
    z_f = ur.urlopen(req)
    f = GzipFile(fileobj=z_f, mode="r")
    od = pd.read_csv(f)
    return od

def get_rac_data(state='ma'):
    req = ur.Request('https://lehd.ces.census.gov/data/lodes/LODES7/{}/rac/ma_rac_S000_JT00_2017.csv.gz'.format(state)) 
    z_f = ur.urlopen(req)
    f = GzipFile(fileobj=z_f, mode="r")
    rac = pd.read_csv(f)
    return rac

def get_wac_data(state='ma'):
    req = ur.Request('https://lehd.ces.census.gov/data/lodes/LODES7/{}/wac/ma_wac_S000_JT00_2017.csv.gz'.format(state)) 
    z_f = ur.urlopen(req)
    f = GzipFile(fileobj=z_f, mode="r")
    wac = pd.read_csv(f)
    return wac

In [21]:
# Origin-Destination (OD) File Structure
# Pos Variable Type Explanation
# 1 w_geocode Char15 Workplace Census Block Code
# 2 h_geocode Char15 Residence Census Block Code
# 3 S000 Num Total number of jobs
# 4 SA01 Num Number of jobs of workers age 29 or younger16
# 5 SA02 Num Number of jobs for workers age 30 to 5416
# 6 SA03 Num Number of jobs for workers age 55 or older16
# 7 SE01 Num Number of jobs with earnings $1250/month or less
# 8 SE02 Num Number of jobs with earnings $1251/month to $3333/month
# 9 SE03 Num Number of jobs with earnings greater than $3333/month
# 10 SI01 Num Number of jobs in Goods Producing industry sectors
# 11 SI02 Num Number of jobs in Trade, Transportation, and Utilities industry sectors
# 12 SI03 Num Number of jobs in All Other Services industry sectors
# 13 createdate Char Date on which data was created, formatted as YYYYMMDD 

In [22]:
od=get_od_data(state='ma')
# block -> block group
od['w_block_group']=od.apply(lambda row: str(row['w_geocode'])[0:12], axis=1)
od['h_block_group']=od.apply(lambda row: str(row['h_geocode'])[0:12], axis=1)
cols_to_sum=['S000', 'SA01', 'SA02', 'SA03', 'SE01','SE02', 'SE03', 'SI01', 'SI02', 'SI03']


In [23]:
# For easy indexing save two copies of the OD matrix df
od_by_hbg_wbg=od.groupby(['h_block_group', 'w_block_group'] , as_index=True)[cols_to_sum].agg('sum')
od_by_wbg_hbg=od.groupby(['w_block_group', 'h_block_group'] , as_index=True)[cols_to_sum].agg('sum')
# also repeat the work bg as a column for doing a secondary indexing (see below)
od_by_hbg_wbg['w_block_group']=od_by_hbg_wbg.apply(lambda row: row.name[1], axis=1)

In [24]:
od_by_hbg_wbg.head()

S000  SA01  SA02  SA03  SE01  SE02  SE03  SI01  \
h_block_group w_block_group                                                   
250010101001  250010101001     30     5    13    12     7    11    12     0   
              250010101002     12     4     4     4     2     7     3     1   
              250010101003     36     3    19    14     7    22     7     0   
              250010101004     62     4    35    23    12    18    32     7   
              250010101005     80    13    35    32    21    37    22     0   

                             SI02  SI03 w_block_group  
h_block_group w_block_group                            
250010101001  250010101001     15    15  250010101001  
              250010101002      1    10  250010101002  
              250010101003      8    28  250010101003  
              250010101004     10    45  250010101004  
              250010101005      8    72  250010101005

In [25]:
od_by_wbg_hbg.head()

S000  SA01  SA02  SA03  SE01  SE02  SE03  SI01  \
w_block_group h_block_group                                                   
250010101001  250010101001     30     5    13    12     7    11    12     0   
              250010101002     18     3     9     6     3     9     6     0   
              250010101003     35     6    13    16     7    10    18     1   
              250010101004     37     5    25     7     6    19    12     0   
              250010101005     23     1    13     9     7     6    10     0   

                             SI02  SI03  
w_block_group h_block_group              
250010101001  250010101001     15    15  
              250010101002      6    12  
              250010101003     16    18  
              250010101004     19    18  
              250010101005      9    14

RAC and WAC fles

In [26]:
rac=get_rac_data(state='ma')
wac=get_wac_data(state='ma')

### Compute metrics for a single GEOID

#### Live-Work Score

In [28]:
bg_geoid='250092033014'
reachable=bg_to_walkable_bg[bg_geoid]
live_reachable=od_by_hbg_wbg.loc[list(reachable)]
work_reachable=od_by_wbg_hbg.loc[list(reachable)]
live_work_reachable=live_reachable.loc[live_reachable['w_block_group'].isin(reachable)]

In [29]:
sum_live_reachable=live_reachable.sum()
sum_work_reachable=work_reachable.sum()
sum_live_work_reachable=live_work_reachable.sum()

use Jaccard Index (Intersection over Union)

https://en.wikipedia.org/wiki/Jaccard_index

In [30]:
jaccard=sum_live_work_reachable['S000']/(
    sum_live_reachable['S000'] + sum_work_reachable['S000'] - sum_live_work_reachable['S000'])
jaccard

0.009880028228652082

#### Similarity of Live and Work Communities

use dot product of vectors for each variable

In [31]:
live_income_vector=sum_live_reachable[['SE01','SE02','SE03']]
work_income_vector=sum_work_reachable[['SE01','SE02','SE03']]
print(live_income_vector)
print(work_income_vector)

SE01    265
SE02    252
SE03    776
dtype: object
SE01    30
SE02    30
SE03    78
dtype: int64


In [32]:
from numpy import dot
from numpy.linalg import norm

In [33]:
a, b = live_income_vector.values, work_income_vector.values
dot(a,b)/(norm(a)*norm(b))

0.998266507763609